# Aggregeringar och analys
Nu när vi gått igenom grunderna och lite mer avancerade koncept för att tvätta och strukturera om data är det nu dags att lära oss hur vi med Pandas kan arbeta med mer analys av datat. Vi kommer att gå igenom

+ Group by-operationer eller split-apply-combine som det ofta kallas
+ Aggregeringar på totalnivå, grupper och över fönster
+ Korstabulleringar och pivottabeller
+ Rullande beräkningar

### Split-Apply-Combine
Det kan vara av värde att kort beskriva det som kallas split-apply-combine eftersom det är ett bra sätt att representera hur data bearbetas i många av de operationer vi kommer att jobba med. Det är också bra att känna till på grund av att Hadoop använder samma dataprocessningsparadigm med sitt map/reduce.

En övergripande beskrivning av split-apply-combine visas i bilden nedan.

<img src="./assets/images/split-apply-combine.png" width="600" align="left">

### Dataimport
Vi börjar med att importera Pandas och lite data att arbeta med.

In [5]:
import pandas as pd

Eftersom vi antagligen kommer vilja analysera datat över tid kan vi ange vilka kolumner som innehåller datum och be pandas försöka göra om dem till datumobjekt.

In [6]:
orders = pd.read_csv('./assets/data/aw_orders.csv', parse_dates=['OrderDate', 'DueDate', 'ShipDate'])

In [7]:
orders.head()

,SalesOrderID,SalesOrderDetailID,OrderDate,DueDate,ShipDate,EmployeeID,CustomerID,SubTotal,TaxAmt,Freight,TotalDue,ProductID,OrderQty,UnitPrice,UnitPriceDiscount,LineTotal
0,43659,1,2011-05-31,2011-06-12,2011-06-07,279,1045,20565.6206,1971.5149,616.0984,23153.2339,776,1,2024.994,0.0,2024.994
1,43659,2,2011-05-31,2011-06-12,2011-06-07,279,1045,20565.6206,1971.5149,616.0984,23153.2339,777,3,2024.994,0.0,6074.982
2,43659,3,2011-05-31,2011-06-12,2011-06-07,279,1045,20565.6206,1971.5149,616.0984,23153.2339,778,1,2024.994,0.0,2024.994
3,43659,4,2011-05-31,2011-06-12,2011-06-07,279,1045,20565.6206,1971.5149,616.0984,23153.2339,771,1,2039.994,0.0,2039.994
4,43659,5,2011-05-31,2011-06-12,2011-06-07,279,1045,20565.6206,1971.5149,616.0984,23153.2339,772,1,2039.994,0.0,2039.994


Genom att titta på datatyperna ser vi att våra kolumner har rätt format. Vi kommer att titta på datumfunktioner mer specifikt längre fram.

In [8]:
orders.dtypes

SalesOrderID                   int64
SalesOrderDetailID             int64
OrderDate             datetime64[ns]
DueDate               datetime64[ns]
ShipDate              datetime64[ns]
EmployeeID                     int64
CustomerID                     int64
SubTotal                     float64
TaxAmt                       float64
Freight                      float64
TotalDue                     float64
ProductID                      int64
OrderQty                       int64
UnitPrice                    float64
UnitPriceDiscount            float64
LineTotal                    float64
dtype: object

### Group By
Den kanske mest använda funktionen inom både SQL och Pandas är `.groupby()`. Vi kommer att använda oss av den här funktionen mycket. När man skapar en groupby i Pandas så returneras ett särskilt objekt för att representera detta.

In [9]:
group = orders.groupby('CustomerID')
group

När vi har detta objekt kan vi applicera en mängd aggregeringsfunktioner på vårt data. Om vi inte gör någon selektering av kolumner kommer funktionen appliceras på alla tillämpbara kolumner.

In [10]:
group.mean().head()

,SalesOrderID,SalesOrderDetailID,EmployeeID,SubTotal,TaxAmt,Freight,TotalDue,ProductID,OrderQty,UnitPrice,UnitPriceDiscount,LineTotal
CustomerID,,,,,,,,,,,,
291,48870.734375,25608.539062,279.000000,30645.294995,2941.029999,919.071866,34505.396861,797.296875,3.500000,359.931763,0.004453,1024.242049
293,61756.833333,76407.703704,276.000000,28828.847678,2831.851646,884.953653,32545.652977,902.648148,3.080247,387.411688,0.008457,696.843620
295,51699.087302,35146.849206,276.058201,45597.307655,4384.448292,1370.140101,51351.896048,795.092593,2.798942,541.527630,0.000212,1374.104117
297,50512.244444,29655.622222,275.666667,9708.571653,1077.634364,336.760738,11122.966756,798.800000,2.133333,833.533253,0.031111,1509.804296
299,61594.255102,76442.974490,285.698980,55207.448735,5480.644581,1712.701415,62400.794731,908.336735,4.974490,408.305455,0.015306,1128.417267


In [11]:
group.sum().head()

,SalesOrderID,SalesOrderDetailID,EmployeeID,SubTotal,TaxAmt,Freight,TotalDue,ProductID,OrderQty,UnitPrice,UnitPriceDiscount,LineTotal
CustomerID,,,,,,,,,,,,
291,6255454,3277893,35712,3.922598e+06,3.764518e+05,117641.1989,4.416691e+06,102054,448,46071.2656,0.57,131102.982305
293,10004607,12378048,44712,4.670273e+06,4.587600e+05,143362.4918,5.272396e+06,146229,499,62760.6935,1.37,112888.666400
295,19542255,13285509,104350,1.723578e+07,1.657321e+06,517912.9582,1.941102e+07,300545,1058,204697.4443,0.08,519411.356320
297,2273051,1334503,12405,4.368857e+05,4.849355e+04,15154.2332,5.005335e+05,35946,96,37508.9964,1.40,67941.193325
299,12072474,14982823,55997,1.082066e+07,1.074206e+06,335689.4773,1.223056e+07,178034,975,80027.8692,3.00,221169.784344


För att enbart arbeta med vissa kolumner kan man göra det.

In [12]:
cols = ['OrderQty', 'LineTotal']

group[cols].sum().head()

,OrderQty,LineTotal
CustomerID,,
291,448,131102.982305
293,499,112888.666400
295,1058,519411.356320
297,96,67941.193325
299,975,221169.784344


Vi kan enkelt gruppera på två eller fler kolumner genom att skicka in en lista på kolumner till groupby-funktionen. Man behöver inte heller deklarera ett `DataFrameGroupBy`-objekt utan kan kedja operationerna som man kan förvänta sig.

Om vi exempelvis vill titta på försäljningen över år och månad behöver vi ta en snabbtitt på delar av det datumfunktioner som Pandas erbjuder. Då Pandas ursprungligen är framtaget inom finansvärlden finns det enorma möjligheter att arbeta med tidsserier. Men vi börjar enkelt med att extrahera år och månadsinformation från kolumnen `OrderDate`.

In [13]:
datum = orders['OrderDate']

In [15]:
print(orders['OrderDate'].dt.year.head())
print(orders['OrderDate'].dt.month.head())

0    2011
1    2011
2    2011
3    2011
4    2011
Name: OrderDate, dtype: int64
0    5
1    5
2    5
3    5
4    5
Name: OrderDate, dtype: int64


Vi kan antingen spara detta i egna variabler och anropa per namn nedan.

In [16]:
orders['OrderYear'] = orders['OrderDate'].dt.year
orders['OrderMonth'] = orders['OrderDate'].dt.month
orders.head()

,SalesOrderID,SalesOrderDetailID,OrderDate,DueDate,ShipDate,EmployeeID,CustomerID,SubTotal,TaxAmt,Freight,TotalDue,ProductID,OrderQty,UnitPrice,UnitPriceDiscount,LineTotal,OrderYear,OrderMonth
0,43659,1,2011-05-31,2011-06-12,2011-06-07,279,1045,20565.6206,1971.5149,616.0984,23153.2339,776,1,2024.994,0.0,2024.994,2011,5
1,43659,2,2011-05-31,2011-06-12,2011-06-07,279,1045,20565.6206,1971.5149,616.0984,23153.2339,777,3,2024.994,0.0,6074.982,2011,5
2,43659,3,2011-05-31,2011-06-12,2011-06-07,279,1045,20565.6206,1971.5149,616.0984,23153.2339,778,1,2024.994,0.0,2024.994,2011,5
3,43659,4,2011-05-31,2011-06-12,2011-06-07,279,1045,20565.6206,1971.5149,616.0984,23153.2339,771,1,2039.994,0.0,2039.994,2011,5
4,43659,5,2011-05-31,2011-06-12,2011-06-07,279,1045,20565.6206,1971.5149,616.0984,23153.2339,772,1,2039.994,0.0,2039.994,2011,5


In [17]:
g = orders.groupby(['OrderYear', 'OrderMonth'])
g['LineTotal'].sum()

OrderYear  OrderMonth
2011       5             4.893286e+05
           7             1.538408e+06
           8             2.010618e+06
           10            4.027080e+06
           12            7.131167e+05
2012       1             3.356069e+06
           2             8.828999e+05
           3             2.269117e+06
           4             1.001804e+06
           5             2.393690e+06
           6             3.601191e+06
           7             2.885359e+06
           8             1.802154e+06
           9             3.053816e+06
           10            2.185213e+06
           11            1.317542e+06
           12            2.384847e+06
2013       1             1.563955e+06
           2             1.865278e+06
           3             2.880753e+06
           4             1.987873e+06
           5             2.667423e+06
           6             4.220928e+06
           7             4.049215e+06
           8             2.284057e+06
           9             3.4

Eller så kan vi göra allting inline med datumfunktioner.

In [18]:
orders.groupby([orders['OrderDate'].dt.year, orders['OrderDate'].dt.month])['LineTotal'].sum()

OrderDate  OrderDate
2011       5            4.893286e+05
           7            1.538408e+06
           8            2.010618e+06
           10           4.027080e+06
           12           7.131167e+05
2012       1            3.356069e+06
           2            8.828999e+05
           3            2.269117e+06
           4            1.001804e+06
           5            2.393690e+06
           6            3.601191e+06
           7            2.885359e+06
           8            1.802154e+06
           9            3.053816e+06
           10           2.185213e+06
           11           1.317542e+06
           12           2.384847e+06
2013       1            1.563955e+06
           2            1.865278e+06
           3            2.880753e+06
           4            1.987873e+06
           5            2.667423e+06
           6            4.220928e+06
           7            4.049215e+06
           8            2.284057e+06
           9            3.486886e+06
           10    

Eftersom vi i det här fallet får ett hierarkiskt index då vi grupperar på två fält kan vi väldigt enkelt pivotera datat med hjälp av `.unstack()`.

In [19]:
df = orders.groupby([orders['OrderDate'].dt.year, orders['OrderDate'].dt.month])['LineTotal'].sum()
df.unstack(1)

OrderDate,1,2,3,4,5,6,7,8,9,10,11,12
OrderDate,,,,,,,,,,,,
2011,NaN,NaN,NaN,NaN,4.893286e+05,NaN,1.538408e+06,2.010618e+06,NaN,4.027080e+06,NaN,7.131167e+05
2012,3.356069e+06,8.828999e+05,2.269117e+06,1.001804e+06,2.393690e+06,3.601191e+06,2.885359e+06,1.802154e+06,3.053816e+06,2.185213e+06,1.317542e+06,2.384847e+06
2013,1.563955e+06,1.865278e+06,2.880753e+06,1.987873e+06,2.667423e+06,4.220928e+06,4.049215e+06,2.284057e+06,3.486886e+06,3.511220e+06,1.668952e+06,2.703811e+06
2014,2.738752e+06,3.230646e+03,5.526352e+06,1.284793e+03,3.415479e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Om vi vill göra flera aggregeringar på vårt data kan vi göra det med funktionen `.aggregate()` och skicka in en lista på de beräkningar vi vill göra. 

In [20]:
g = orders.groupby(['OrderYear', 'OrderMonth'])
g.aggregate(['sum', 'mean']).head()

SalesOrderID               SalesOrderDetailID  \
                              sum          mean                sum   
OrderYear OrderMonth                                                 
2011      5              15374439  43677.383523              62128   
          7              34447814  43882.565605             699435   
          8              48260663  44194.746337            2241020   
          10             85198924  44630.133054            7376544   
          12             16986736  45057.655172            2013180   

                                  EmployeeID             CustomerID  \
                             mean        sum        mean        sum   
OrderYear OrderMonth                                                  
2011      5            176.500000      98388  279.511364     323128   
          7            891.000000     218525  278.375796     788751   
          8           2052.216117     304392  278.747253    1001590   
          10          3864.088004     531509  278.422734    1909601   
          12          5340.000000     105234  279.135279     352439   

                                       SubTotal                   ...       \
                             mean           sum          mean     ...        
OrderYear OrderMonth                                              ...        
2011      5            917.977273  7.174123e+06  20381.031874     ...        
          7           1004.778344  2.265582e+07  28860.918219     ...        
          8            917.206960  3.327216e+07  30469.014808     ...        
          10          1000.314825  7.557602e+07  39589.325540     ...        
          12           934.851459  9.164881e+06  24310.028346     ...        

                     ProductID             OrderQty               UnitPrice  \
                           sum        mean      sum      mean           sum   
OrderYear OrderMonth                                                          
2011      5             262249  745.025568      820  2.329545  2.307668e+05   
          7             586108  746.634395     2053  2.615287  5.957644e+05   
          8             814631  745.999084     2754  2.521978  7.779667e+05   
          10           1424546  746.226296     5208  2.728130  1.421921e+06   
          12            283817  752.830239      852  2.259947  3.049708e+05   

                                 UnitPriceDiscount               LineTotal  \
                            mean               sum      mean           sum   
OrderYear OrderMonth                                                         
2011      5           655.587431              0.00  0.000000  4.893286e+05   
          7           758.935565              0.27  0.000344  1.538408e+06   
          8           712.423714              0.25  0.000229  2.010618e+06   
          10          744.850998              0.43  0.000225  4.027080e+06   
          12          808.941225              0.02  0.000053  7.131167e+05   

                                   
                             mean  
OrderYear OrderMonth               
2011      5           1390.138008  
          7           1959.755812  
          8           1841.225343  
          10          2109.523489  
          12          1891.556218  

[5 rows x 26 columns]

Vi kan också skicka med en dict med `kolumn : aggregeringsfunktion` om vi vill göra individuella aggregeringar per kolumn.

In [21]:
g.aggregate({'LineTotal' : 'mean', 'OrderQty' : 'sum'}).head()

LineTotal  OrderQty
OrderYear OrderMonth                       
2011      5           1390.138008       820
          7           1959.755812      2053
          8           1841.225343      2754
          10          2109.523489      5208
          12          1891.556218       852

Vi kan också göra flera beräkningar på vissa kolumner genom att skicka in en dict med en lista som värde likt nedan.

In [22]:
g.aggregate({'LineTotal' : 'mean', 'OrderQty' : ['sum', 'mean']}).head()

LineTotal OrderQty          
                             mean      sum      mean
OrderYear OrderMonth                                
2011      5           1390.138008      820  2.329545
          7           1959.755812     2053  2.615287
          8           1841.225343     2754  2.521978
          10          2109.523489     5208  2.728130
          12          1891.556218      852  2.259947

Som vi sett hittills skapas nya rad och kolumnindex när man arbetar med `.groupby()`. Det beteendet går att undvika genom att ange `as_index=False` så här.

In [23]:
orders.groupby(['OrderYear', 'OrderMonth'], as_index=False)['OrderQty', 'LineTotal'].sum().head()

,OrderYear,OrderMonth,OrderQty,LineTotal
0,2011,5,820,4.893286e+05
1,2011,7,2053,1.538408e+06
2,2011,8,2754,2.010618e+06
3,2011,10,5208,4.027080e+06
4,2011,12,852,7.131167e+05


Eftersom Pandas returnerar en `DataFrame` när vi gör en aggregeringsfunktion kan vi göra fortsatta beräkningar mellan kolumner. Exempelvis räkna ut snittpris per produkt per år.

In [24]:
data = orders.groupby('OrderYear')['OrderQty', 'LineTotal'].sum()
data['AveragePrice'] = data['LineTotal'] / data['OrderQty']
data

,OrderQty,LineTotal,AveragePrice
OrderYear,,,
2011,11687,8.778552e+06,751.138188
2012,65836,2.713370e+07,412.140795
2013,102829,3.289035e+07,319.854824
2014,34164,1.168510e+07,342.029595


Vi kan också använda oss av funktionen `.describe()` för gruppvis explorativ analys.

In [25]:
orders.groupby('OrderYear').describe()

CustomerID    EmployeeID       Freight     LineTotal  \
OrderYear                                                                 
2011      count   4515.000000   4515.000000   4515.000000   4515.000000   
          mean     969.104983    278.637431    984.653048   1944.308306   
          std      446.911235      2.557945    744.554739   2991.355784   
          min      291.000000    274.000000      0.154400      5.186500   
          25%      647.000000    277.000000    463.895400    183.938200   
          50%      883.000000    279.000000    833.184200    838.917800   
          75%     1239.000000    281.000000   1267.107100   2145.537450   
          max     1991.000000    283.000000   3836.639000  27055.760420   
2012      count  18946.000000  18946.000000  18946.000000  18946.000000   
          mean    1052.497625    279.991449   1373.829292   1432.159895   
          std      488.391503      4.754499    909.301545   2120.259749   
          min      291.000000    274.000000      0.382700      5.186500   
          25%      663.000000    276.000000    736.832000    145.788000   
          50%      977.000000    279.000000   1285.860000    600.262500   
          75%     1335.000000    282.000000   1829.014400   1879.176000   
          max     1993.000000    290.000000   4684.578600  23020.131790   
2013      count  27614.000000  27614.000000  27614.000000  27614.000000   
          mean    1057.700804    281.355110   1289.559002   1191.075242   
          std      488.744990      5.256151    883.970772   1982.529158   
          min      291.000000    274.000000      0.034400      1.374000   
          25%      651.000000    277.000000    761.070500    114.300000   
          50%     1001.000000    280.000000   1143.486300    404.664000   
          75%     1342.500000    287.000000   1731.331600   1409.382000   
          max     1993.000000    290.000000   5608.912100  27893.619000   
2014      count   9844.000000   9844.000000   9844.000000   9844.000000   
          mean    1039.436814    281.612353   1130.186575   1187.027538   
          std      492.098514      5.187383    725.829721   1963.583672   
          min      293.000000    274.000000      0.068700      1.374000   
          25%      625.000000    277.000000    705.692200    109.884000   
          50%      985.000000    281.000000    988.504300    404.664000   
          75%     1323.000000    286.000000   1592.887000   1427.592000   
          max     1993.000000    290.000000   3512.671900  19186.116400   

                   OrderMonth      OrderQty     ProductID  SalesOrderDetailID  \
OrderYear                                                                       
2011      count   4515.000000   4515.000000   4515.000000         4515.000000   
          mean       8.771872      2.588483    746.700111         2744.697010   
          std        1.818713      1.991531     23.384706         1590.785046   
          min        5.000000      1.000000    707.000000            1.000000   
          25%        7.000000      1.000000    725.000000         1275.500000   
          50%       10.000000      2.000000    754.000000         2867.000000   
          75%       10.000000      3.000000    766.000000         4164.500000   
          max       12.000000     26.000000    778.000000         5528.000000   
2012      count  18946.000000  18946.000000  18946.000000        18946.000000   
          mean       6.928481      3.474929    781.481896        16677.582814   
          std        3.130471      2.816557     46.157311         6204.648559   
          min        1.000000      1.000000    707.000000         5717.000000   
          25%        5.000000      2.000000    748.000000        11458.250000   
          50%        7.000000      3.000000    779.000000        16728.500000   
          75%        9.000000      4.000000    819.000000        21891.750000   
          max       12.000000     39.000000    863.000000        27395.000000   
2013      

Förutom aggregeringar kan vi även på ett smidigt sätt applicera gruppvisa transformeringar på vårt data genom funktionen `.transform()`. I exemplet nedan skapar vi en funktion för att räkna ut zscore och applicerar på våra kolumner.

In [26]:
cols = ['OrderQty', 'LineTotal']
zscore = lambda x: (x - x.mean()) / x.std()

g[cols].transform(zscore).head()

,OrderQty,LineTotal
0,-0.852483,0.325521
1,0.429884,2.402142
2,-0.852483,0.325521
3,-0.852483,0.333212
4,-0.852483,0.333212


Nu när vi har gått igenom grunderna kan vi börja producera mer avancerade analyser. Vi kan exempelvis titta på köp per kund under 2012 såhär.

In [27]:
orders.head()

,SalesOrderID,SalesOrderDetailID,OrderDate,DueDate,ShipDate,EmployeeID,CustomerID,SubTotal,TaxAmt,Freight,TotalDue,ProductID,OrderQty,UnitPrice,UnitPriceDiscount,LineTotal,OrderYear,OrderMonth
0,43659,1,2011-05-31,2011-06-12,2011-06-07,279,1045,20565.6206,1971.5149,616.0984,23153.2339,776,1,2024.994,0.0,2024.994,2011,5
1,43659,2,2011-05-31,2011-06-12,2011-06-07,279,1045,20565.6206,1971.5149,616.0984,23153.2339,777,3,2024.994,0.0,6074.982,2011,5
2,43659,3,2011-05-31,2011-06-12,2011-06-07,279,1045,20565.6206,1971.5149,616.0984,23153.2339,778,1,2024.994,0.0,2024.994,2011,5
3,43659,4,2011-05-31,2011-06-12,2011-06-07,279,1045,20565.6206,1971.5149,616.0984,23153.2339,771,1,2039.994,0.0,2039.994,2011,5
4,43659,5,2011-05-31,2011-06-12,2011-06-07,279,1045,20565.6206,1971.5149,616.0984,23153.2339,772,1,2039.994,0.0,2039.994,2011,5


In [28]:
f = orders['OrderYear'] == 2012

customer_sales_2012 = orders[f].groupby('CustomerID')['OrderQty', 'LineTotal'].sum()
customer_sales_2012.head()

,OrderQty,LineTotal
CustomerID,,
291,268,65177.167955
295,388,230735.677500
297,37,22003.889425
301,68,30518.974300
303,12,6181.791300


Nu kan vi räkna ut decilerna och addera till datasetet. Genom att sätta `labels=False` får vi istället en serie mellan 0 och 9 vilket passar bra för ändamålet.

In [29]:
customer_sales_2012['Decile'] = pd.qcut(customer_sales_2012['LineTotal'], 10, labels=False)
customer_sales_2012.head()

,OrderQty,LineTotal,Decile
CustomerID,,,
291,268,65177.167955,6
295,388,230735.677500,9
297,37,22003.889425,4
301,68,30518.974300,5
303,12,6181.791300,3


Nu när vi har lagt på vår deciluppdelning kan vi analysera datasetet.

In [30]:
final = customer_sales_2012.groupby('Decile')['LineTotal'].describe()
final

Decile       
0       count        42.000000
        mean        502.256009
        std         315.542691
        min          20.520000
        25%         188.536650
        50%         498.634400
        75%         777.312150
        max        1021.328300
1       count        42.000000
        mean       1717.204587
        std         478.108054
        min        1041.249800
        25%        1309.303325
        50%        1584.058500
        75%        2248.070750
        max        2505.890100
2       count        42.000000
        mean       3854.827248
        std         984.193746
        min        2554.275800
        25%        2911.439425
        50%        3737.470250
        75%        4730.579025
        max        5757.732500
3       count        42.000000
        mean       9275.002427
        std        2703.818109
        min        5788.910200
        25%        6630.676925
        50%        9194.740900
                     ...      
6       std       10891.7

För att göra det lite mer lättläsligt kan vi pivotera vårt index med `.unstack()`.

In [31]:
final.unstack()

,count,mean,std,min,25%,50%,75%,max
Decile,,,,,,,,
0,42.0,502.256009,315.542691,20.520000,188.536650,498.634400,777.312150,1021.328300
1,42.0,1717.204587,478.108054,1041.249800,1309.303325,1584.058500,2248.070750,2505.890100
2,42.0,3854.827248,984.193746,2554.275800,2911.439425,3737.470250,4730.579025,5757.732500
3,42.0,9275.002427,2703.818109,5788.910200,6630.676925,9194.740900,11339.923825,14149.943600
4,42.0,20383.742295,3728.631120,14418.457500,16431.928425,20831.916162,23933.085772,25671.027300
5,41.0,34559.345405,5503.122489,25885.464300,30403.936525,34130.771900,39125.410700,45051.980700
6,42.0,63565.814805,10891.759660,46004.220475,52032.236425,65566.640879,72255.310538,80377.433600
7,42.0,102104.296050,10012.234544,80727.052300,94849.307150,103371.250834,109930.878175,117541.364208
8,42.0,150381.902112,18873.528321,122769.296400,137185.873794,148008.447200,163384.680970,186628.455000


För att titta på varje decils bidrag till total försäljning gör vi det enkelt genom räkna ut total försäljning och sedan applicera en summering per grupp dividerat med den totala försäljningen. Som synes står översta decilen för 40,3% av den totala försäljningen.

In [32]:
total_sales = customer_sales_2012['LineTotal'].sum()

customer_sales_2012.groupby('Decile')['LineTotal'].apply(lambda x: x.sum() / total_sales * 100)

Decile
0     0.077744
1     0.265804
2     0.596685
3     1.435669
4     3.155180
5     5.222042
6     9.839292
7    15.804628
8    23.277473
9    40.325484
Name: LineTotal, dtype: float64

### Pivottabeller 
Med `.pivot_table()` i Pandas får vi ett gränssnitt för att enkelt skapa pivottabeller likt excel. Funktionaliteten är ganska straight forward.

In [33]:
orders.head()

,SalesOrderID,SalesOrderDetailID,OrderDate,DueDate,ShipDate,EmployeeID,CustomerID,SubTotal,TaxAmt,Freight,TotalDue,ProductID,OrderQty,UnitPrice,UnitPriceDiscount,LineTotal,OrderYear,OrderMonth
0,43659,1,2011-05-31,2011-06-12,2011-06-07,279,1045,20565.6206,1971.5149,616.0984,23153.2339,776,1,2024.994,0.0,2024.994,2011,5
1,43659,2,2011-05-31,2011-06-12,2011-06-07,279,1045,20565.6206,1971.5149,616.0984,23153.2339,777,3,2024.994,0.0,6074.982,2011,5
2,43659,3,2011-05-31,2011-06-12,2011-06-07,279,1045,20565.6206,1971.5149,616.0984,23153.2339,778,1,2024.994,0.0,2024.994,2011,5
3,43659,4,2011-05-31,2011-06-12,2011-06-07,279,1045,20565.6206,1971.5149,616.0984,23153.2339,771,1,2039.994,0.0,2039.994,2011,5
4,43659,5,2011-05-31,2011-06-12,2011-06-07,279,1045,20565.6206,1971.5149,616.0984,23153.2339,772,1,2039.994,0.0,2039.994,2011,5


In [34]:
orders.pivot_table(values=['OrderQty', 'LineTotal'], index='EmployeeID', columns='UnitPriceDiscount', aggfunc='mean').stack(0)

UnitPriceDiscount            0.00         0.02         0.05         0.10  \
EmployeeID                                                                 
274        OrderQty      3.494580    12.695652    17.875000          NaN   
           LineTotal  1362.700835  1493.647400  1995.844372          NaN   
275        OrderQty      3.031790    12.076087    17.794872    12.296296   
           LineTotal  1307.630444  2662.116128  1501.026179   228.532170   
276        OrderQty      3.307727    12.303371    17.942857    24.545455   
           LineTotal  1426.137295  2596.997969  1682.656310  2415.531215   
277        OrderQty      3.090278    12.287671    18.048193    16.523810   
           LineTotal  1293.422761  1372.636623  1381.865826   335.514103   
278        OrderQty      3.081011    12.457447    17.000000    12.800000   
           LineTotal  1081.301304  1855.183112  1148.515636   214.520058   
279        OrderQty      2.765227    12.250000    18.093750    11.523810   
           LineTotal  1325.987018  2207.214974   449.443815   209.072816   
280        OrderQty      3.182099    12.000000    16.928571    17.000000   
           LineTotal  1602.866536  2706.660859  1825.849969   386.376150   
281        OrderQty      3.059273    12.296875    17.750000    14.312500   
           LineTotal  1358.366888  2910.704673  4487.117081  2007.740711   
282        OrderQty      3.032404    12.048544    17.477273    16.666667   
           LineTotal  1311.763011  2260.693576  1836.292445   464.954430   
283        OrderQty      3.237328    12.142857    18.066667    16.100000   
           LineTotal  1608.213416  4363.701807  1059.328787   420.499647   
284        OrderQty      2.862562    12.117647    16.800000          NaN   
           LineTotal  1214.135575  2248.403416   474.551220          NaN   
285        OrderQty      3.155340    12.333333    19.000000          NaN   
           LineTotal   692.472553   426.678933   402.544450          NaN   
286        OrderQty      2.660767    11.941176    18.500000          NaN   
           LineTotal   953.083323  1760.138535   431.234925          NaN   
287        OrderQty      3.031193    12.500000    16.375000          NaN   
           LineTotal  1253.223073  1673.935130   474.996913          NaN   
288        OrderQty      3.299688    12.216667    17.545455    30.333333   
           LineTotal   994.265400  1476.302857   967.722250  1112.550000   
289        OrderQty      3.408763    12.397959    17.435897    16.875000   
           LineTotal  1245.569565  1688.121176  1715.795575   407.332125   
290        OrderQty      3.559911    12.245614    17.473684     8.909091   
           LineTotal  1255.499873  2507.815175  2417.894634   264.333641   

UnitPriceDiscount            0.15         0.20         0.30         0.35  \
EmployeeID                                                                 
274        OrderQty      2.000000     1.000000          NaN     6.875000   
           LineTotal   567.897750   762.902400          NaN  3786.238828   
275        OrderQty      3.000000     3.111111     4.333333     2.250000   
           LineTotal   495.839783  2373.474133   712.520900  1238.043219   
276        OrderQty      5.170213     3.578947     5.000000     4.333333   
           LineTotal   807.129120  2730.387537   822.139500  2383.713196   
277        OrderQty      3.235294     2.000000     6.833333     2.444444   
           LineTotal   446.495325  1525.804800  1123.590650  1345.586306   
278        OrderQty      2.800000     2.000000     7.200000     2.000000   
           LineTotal   659.774250  1525.804800  1183.880880  1099.580083   
279        OrderQty      3.372093     2.666667     3.714286     2.625000   
           LineTotal   611.430253  2034.406400   610.732200  1445.060839   
280        OrderQty      3.280000     2.555556     3.000000          NaN   
           LineTotal   931.352310  1949.639467   493.283700          NaN   
281        OrderQty      4.795918     3.529412 

Vi kan även skapa hierarkiska index för både rader och kolumner. I det här fallet år/månad på våra kolumner.

In [35]:
orders.pivot_table(values='LineTotal', index='ProductID', columns=['OrderYear', 'OrderMonth'], aggfunc='sum').head()

OrderYear       2011                                                 \
OrderMonth        5           7           8            10        12   
ProductID                                                             
707         484.4760  1170.81700  1937.90400  2846.296500  242.2380   
708         545.0355  1130.44400  1736.03900  3270.213000  201.8650   
709         216.6000   711.28875   895.00925  1234.791950  256.5000   
710          28.5000    74.10000   108.30000   165.300000       NaN   
711         666.1545  1291.93600  1513.98750  3642.061608  141.3055   

OrderYear         2012                                               \
OrderMonth          1         2            3         4           5    
ProductID                                                             
707         1231.37650  545.0355  1877.344500  1049.698  2452.66218   
708         1312.12250  565.2220  1843.336764  1493.801  2316.40317   
709         1009.30565  427.7451   756.247500   552.900         NaN   
710                NaN   34.2000    79.800000    22.800         NaN   
711         1816.78500  666.1545  1857.158000  1736.039  2679.76053   

OrderYear      ...              2013                                      \
OrderMonth     ...                8            9            10        11   
ProductID      ...                                                         
707            ...       3120.324224  3790.760616  3631.934008  1553.556   
708            ...       3216.655193  3997.988891  4417.868891  1721.508   
709            ...               NaN          NaN          NaN       NaN   
710            ...               NaN          NaN          NaN       NaN   
711            ...       3031.159207  3925.556092  4384.418451  1931.448   

OrderYear                 2014                                            
OrderMonth        12        1       2            3       4            5   
ProductID                                                                 
707         1259.640  1532.562     NaN  7078.445509     NaN  3547.958008  
708         2288.346  2435.304     NaN  7240.326744     NaN  4276.411319  
709              NaN       NaN     NaN          NaN     NaN          NaN  
710              NaN       NaN     NaN          NaN     NaN          NaN  
711         2057.412  2141.388  41.988  7460.809231  41.988  4143.428325  

[5 rows x 34 columns]

### Cross-tabulation
För att beräkna korstabeller kan funktionen `.crosstab()` användas. Fungerar som pivottabell.

In [36]:
products = pd.read_csv('./assets/data/aw_products.csv')
products.head()

,ProductID,ProductNumber,ProductName,ModelName,MakeFlag,StandardCost,ListPrice,SubCategoryID
0,680,FR-R92B-58,"HL Road Frame - Black, 58",HL Road Frame,1,1059.3100,1431.50,14
1,706,FR-R92R-58,"HL Road Frame - Red, 58",HL Road Frame,1,1059.3100,1431.50,14
2,707,HL-U509-R,"Sport-100 Helmet, Red",Sport-100,0,13.0863,34.99,31
3,708,HL-U509,"Sport-100 Helmet, Black",Sport-100,0,13.0863,34.99,31
4,709,SO-B909-M,"Mountain Bike Socks, M",Mountain Bike Socks,0,3.3963,9.50,23


In [37]:
pd.crosstab(products['SubCategoryID'], products['MakeFlag']).head()

MakeFlag,0,1
SubCategoryID,,
1,0,32
2,0,43
3,0,22
4,0,8
5,0,3


### Dates and time-series
Python och Pandas har en mängd verktyg för bearbetning av datum och tid. Den här sektionen kommer att täcka de grundläggande funktionerna och dess tillämpningsområden.

Vi börjar med att titta på Pythons inbyggda datumbibliotek.

In [38]:
from datetime import datetime

Vi kan skapa ett tidsobjekt genom funktionen nedan. 

In [39]:
now = datetime.now()
now

datetime.datetime(2016, 11, 6, 14, 17, 34, 545682)

Från detta objekt kan vi sedan ta ut de beståndsdelar vi är intresserade av.

In [40]:
print(now.year)
print(now.month)
print(now.day)
print(now.hour)
print(now.minute)

2016
11
6
14
17


Vi kan skapa ett tidsobjekt enligt nedan. Om man tar ett tidsobjekt minus ett annat returneras ett objekt av typen timedelta.

In [42]:
birth = datetime(1975, 6, 13)

lifetime = datetime.now() - birth
lifetime

datetime.timedelta(15122, 51492, 397067)

Vi kan ta ut deltat i antal dagar från deltaobjektet så här.

In [43]:
lifetime.days / 365

41.43013698630137

Vi kan också skapa ett deltaobjekt och använda för beräkningar.

In [44]:
from datetime import timedelta

In [45]:
delta = timedelta(14)

datetime.now() + delta

datetime.datetime(2016, 11, 20, 14, 18, 17, 952086)

#### konvertera mellan datum och text
För att konvertera från en textsträng till ett datumobjekt kan funktionen `.strptime()` användas. 

In [46]:
datum = '2016-04-07'

datum_objekt = datetime.strptime(datum, '%Y-%m-%d')
datum_objekt

datetime.datetime(2016, 4, 7, 0, 0)

Funktionen kräver koder för hur den ska tolka datumsträngen. I tabellen nedan visas de koder som finns tillgängliga.

<img src="./assets/images/dates1.png" width="500" align="left">
<img src="./assets/images/dates2.png" width="500" align="left">

För att konvertera ett datumobjekt till text så kan funktionen `.strftime()` användas. Den kräver också en formatmall enligt ovan.

In [47]:
datum_objekt.strftime('%d/%m/%y')

'07/04/16'

#### hantering av tid och tidsserier i Pandas
Pandas är ursprungligen utvecklat inom finansbranschen där tidsserieanalys är väldigt viktigt. Följaktligen har Pandas mycket bra stöd för den här typen av analyser. Vi kan läsa in vår tabell med ordrar igen men utan att parsa datumen den här gången.

In [48]:
data = pd.read_csv('./assets/data/aw_orders.csv')

In [49]:
data.head()

,SalesOrderID,SalesOrderDetailID,OrderDate,DueDate,ShipDate,EmployeeID,CustomerID,SubTotal,TaxAmt,Freight,TotalDue,ProductID,OrderQty,UnitPrice,UnitPriceDiscount,LineTotal
0,43659,1,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,776,1,2024.994,0.0,2024.994
1,43659,2,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,777,3,2024.994,0.0,6074.982
2,43659,3,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,778,1,2024.994,0.0,2024.994
3,43659,4,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,771,1,2039.994,0.0,2039.994
4,43659,5,5/31/2011,6/12/2011,6/7/2011,279,1045,20565.6206,1971.5149,616.0984,23153.2339,772,1,2039.994,0.0,2039.994


Om vi tittar på datatyperna så ser vi att kolumnen `OrderDate` är av typen object vilket är Pandas definition av text.

In [50]:
data.dtypes

SalesOrderID            int64
SalesOrderDetailID      int64
OrderDate              object
DueDate                object
ShipDate               object
EmployeeID              int64
CustomerID              int64
SubTotal              float64
TaxAmt                float64
Freight               float64
TotalDue              float64
ProductID               int64
OrderQty                int64
UnitPrice             float64
UnitPriceDiscount     float64
LineTotal             float64
dtype: object

Vi vill konvertera kolumnen till formatet `datetime` vilket vi kan göra på olika sätt. Om vi håller oss till standardverktygen kan vi använda `.apply()` och `.strptime()` enligt nedan.

In [51]:
datumnkolumner = ['OrderDate', 'DueDate', 'ShipDate']

for col in datumnkolumner:
    data[col] = data[col].apply(lambda x: datetime.strptime(x, '%m/%d/%Y'))

data.dtypes

SalesOrderID                   int64
SalesOrderDetailID             int64
OrderDate             datetime64[ns]
DueDate               datetime64[ns]
ShipDate              datetime64[ns]
EmployeeID                     int64
CustomerID                     int64
SubTotal                     float64
TaxAmt                       float64
Freight                      float64
TotalDue                     float64
ProductID                      int64
OrderQty                       int64
UnitPrice                    float64
UnitPriceDiscount            float64
LineTotal                    float64
dtype: object

In [52]:
data.head()

,SalesOrderID,SalesOrderDetailID,OrderDate,DueDate,ShipDate,EmployeeID,CustomerID,SubTotal,TaxAmt,Freight,TotalDue,ProductID,OrderQty,UnitPrice,UnitPriceDiscount,LineTotal
0,43659,1,2011-05-31,2011-06-12,2011-06-07,279,1045,20565.6206,1971.5149,616.0984,23153.2339,776,1,2024.994,0.0,2024.994
1,43659,2,2011-05-31,2011-06-12,2011-06-07,279,1045,20565.6206,1971.5149,616.0984,23153.2339,777,3,2024.994,0.0,6074.982
2,43659,3,2011-05-31,2011-06-12,2011-06-07,279,1045,20565.6206,1971.5149,616.0984,23153.2339,778,1,2024.994,0.0,2024.994
3,43659,4,2011-05-31,2011-06-12,2011-06-07,279,1045,20565.6206,1971.5149,616.0984,23153.2339,771,1,2039.994,0.0,2039.994
4,43659,5,2011-05-31,2011-06-12,2011-06-07,279,1045,20565.6206,1971.5149,616.0984,23153.2339,772,1,2039.994,0.0,2039.994


Ett smidigare sätt är att använda Pandas egna inbyggda parser. Som vi ser nedanför så har Pandas inga problem att göra om `OrderDate` till datumformat. Däremot är operationen inte lika effektiv då den behöver tolka formatet innan den parsar.

In [53]:
data['OrderDate'] = pd.to_datetime(data['OrderDate'])
data.dtypes

SalesOrderID                   int64
SalesOrderDetailID             int64
OrderDate             datetime64[ns]
DueDate               datetime64[ns]
ShipDate              datetime64[ns]
EmployeeID                     int64
CustomerID                     int64
SubTotal                     float64
TaxAmt                       float64
Freight                      float64
TotalDue                     float64
ProductID                      int64
OrderQty                       int64
UnitPrice                    float64
UnitPriceDiscount            float64
LineTotal                    float64
dtype: object

För att titta på den grundläggande funktionaliteten skapar vi ett dataset med `Order Date` som index och summerar försäljningen per datum.

In [54]:
ts = data.groupby('OrderDate')['LineTotal'].sum()
ts.head()

OrderDate
2011-05-31    4.893286e+05
2011-07-01    1.538408e+06
2011-08-01    1.165897e+06
2011-08-31    8.447210e+05
2011-10-01    2.324136e+06
Name: LineTotal, dtype: float64

Om vi tittar på vårt index nu ser vi att det är av typen `DatetimeIndex`.

In [55]:
ts.index

DatetimeIndex(['2011-05-31', '2011-07-01', '2011-08-01', '2011-08-31',
               '2011-10-01', '2011-10-31', '2011-12-01', '2012-01-01',
               '2012-01-29', '2012-02-29', '2012-03-30', '2012-04-30',
               '2012-05-30', '2012-06-30', '2012-07-31', '2012-08-30',
               '2012-09-30', '2012-10-30', '2012-11-30', '2012-12-31',
               '2013-01-28', '2013-02-28', '2013-03-30', '2013-04-30',
               '2013-05-30', '2013-06-30', '2013-07-31', '2013-08-30',
               '2013-09-30', '2013-10-30', '2013-11-30', '2013-12-31',
               '2014-01-28', '2014-01-29', '2014-02-28', '2014-03-01',
               '2014-03-30', '2014-03-31', '2014-04-30', '2014-05-01'],
              dtype='datetime64[ns]', name='OrderDate', freq=None)

När vi har den här typen av index får vi utökad funktionalitet för att titta på specifika tidsintervall.

In [56]:
ts['2011']

OrderDate
2011-05-31    4.893286e+05
2011-07-01    1.538408e+06
2011-08-01    1.165897e+06
2011-08-31    8.447210e+05
2011-10-01    2.324136e+06
2011-10-31    1.702945e+06
2011-12-01    7.131167e+05
Name: LineTotal, dtype: float64

In [57]:
ts['2011-07']

OrderDate
2011-07-01    1.538408e+06
Name: LineTotal, dtype: float64

In [58]:
ts['2011':'2012']

OrderDate
2011-05-31    4.893286e+05
2011-07-01    1.538408e+06
2011-08-01    1.165897e+06
2011-08-31    8.447210e+05
2011-10-01    2.324136e+06
2011-10-31    1.702945e+06
2011-12-01    7.131167e+05
2012-01-01    1.900789e+06
2012-01-29    1.455280e+06
2012-02-29    8.828999e+05
2012-03-30    2.269117e+06
2012-04-30    1.001804e+06
2012-05-30    2.393690e+06
2012-06-30    3.601191e+06
2012-07-31    2.885359e+06
2012-08-30    1.802154e+06
2012-09-30    3.053816e+06
2012-10-30    2.185213e+06
2012-11-30    1.317542e+06
2012-12-31    2.384847e+06
Name: LineTotal, dtype: float64

In [59]:
ts['2011-10':'2012']

OrderDate
2011-10-01    2.324136e+06
2011-10-31    1.702945e+06
2011-12-01    7.131167e+05
2012-01-01    1.900789e+06
2012-01-29    1.455280e+06
2012-02-29    8.828999e+05
2012-03-30    2.269117e+06
2012-04-30    1.001804e+06
2012-05-30    2.393690e+06
2012-06-30    3.601191e+06
2012-07-31    2.885359e+06
2012-08-30    1.802154e+06
2012-09-30    3.053816e+06
2012-10-30    2.185213e+06
2012-11-30    1.317542e+06
2012-12-31    2.384847e+06
Name: LineTotal, dtype: float64

Vi kan även filtrera på datumintervall som inte finns i indexet.

In [60]:
ts['2011-10-15':'2012-08-10']

OrderDate
2011-10-31    1.702945e+06
2011-12-01    7.131167e+05
2012-01-01    1.900789e+06
2012-01-29    1.455280e+06
2012-02-29    8.828999e+05
2012-03-30    2.269117e+06
2012-04-30    1.001804e+06
2012-05-30    2.393690e+06
2012-06-30    3.601191e+06
2012-07-31    2.885359e+06
Name: LineTotal, dtype: float64

#### date ranges, frequency & shifting
Vårt data innehåller nu lite slumpmässiga datum med olika intervall. Oftast inget problem men om vi skulle vilja justera vårt index till att hålla en observation per datum kan vi enkelt göra det.

In [61]:
ts.head()

OrderDate
2011-05-31    4.893286e+05
2011-07-01    1.538408e+06
2011-08-01    1.165897e+06
2011-08-31    8.447210e+05
2011-10-01    2.324136e+06
Name: LineTotal, dtype: float64

In [63]:
ts.resample('D').mean().head()

OrderDate
2011-05-31    489328.5787
2011-06-01            NaN
2011-06-02            NaN
2011-06-03            NaN
2011-06-04            NaN
Freq: D, Name: LineTotal, dtype: float64

Som vi ser skapar Pandas upp ett index per dag. De dagar där vi inte har någon försäljning hanteras istället som null-värden. Om vi istället vill indexera om till månader justeras alla summor till månadsslut och månader med flera observationer summeras.

In [64]:
ts['2011-08']

OrderDate
2011-08-01    1.165897e+06
2011-08-31    8.447210e+05
Name: LineTotal, dtype: float64

In [69]:
ts.resample('M').agg(['mean', 'sum']).head()

,mean,sum
OrderDate,,
2011-05-31,4.893286e+05,4.893286e+05
2011-06-30,NaN,NaN
2011-07-31,1.538408e+06,1.538408e+06
2011-08-31,1.005309e+06,2.010618e+06
2011-09-30,NaN,NaN


För att arbeta olika tidsintervall har pandas en funktion som heter `.daterange()` som är kompetent för att räkna ut en mängd olika intervall och beräkningar på tid. Dessa intervall kan sedan användas som filter för beräkningar. I dess enklaste form ser det ut så här.

In [70]:
f = pd.date_range(start='2012-01-01', end='2012-06-30')
ts[f].sum()

13504770.000985995

Istället för ett fast intervall kan vi ange perioder framåt.

In [71]:
f = pd.date_range(start='2012-01-01', periods=60)
ts[f].sum()

4238969.286337997

Eller perioder bakåt.

In [72]:
f = pd.date_range(end='2012-01-01', periods=60)
ts[f].sum()

2613905.6246979977

In [74]:
intervall = [30,60,90,180,365]

for i in intervall:
    f = pd.date_range(end='2012-12-31', periods=i)
    print(ts[f].sum())

2384846.590842996
3702388.424246991
5887601.639059987
13628931.376122987
25232912.446750987


Default för perioder är dagar men den går att ange andra frekvenser beroende på vad man är ute efter. I det här fallet vill vi titta på 6 veckors data bakåt från `2012-12-31`. Vilket vi kan göra på följande sätt.

In [75]:
f = pd.date_range(end='2012-12-31', periods=6, freq='W-MON')
ts[f]

2012-11-26             NaN
2012-12-03             NaN
2012-12-10             NaN
2012-12-17             NaN
2012-12-24             NaN
2012-12-31    2.384847e+06
Freq: W-MON, Name: LineTotal, dtype: float64

Här ser vi dock att vi inte får med försäljningen för `2012-11-30`. 

In [76]:
ts['2012']

OrderDate
2012-01-01    1.900789e+06
2012-01-29    1.455280e+06
2012-02-29    8.828999e+05
2012-03-30    2.269117e+06
2012-04-30    1.001804e+06
2012-05-30    2.393690e+06
2012-06-30    3.601191e+06
2012-07-31    2.885359e+06
2012-08-30    1.802154e+06
2012-09-30    3.053816e+06
2012-10-30    2.185213e+06
2012-11-30    1.317542e+06
2012-12-31    2.384847e+06
Name: LineTotal, dtype: float64

Detta kan vi enkelt åtgärda genom att använda oss av `.resample()` och sedan lägga på vårt önskade intervall.

In [77]:
ts.resample('W-MON').sum()[f]

2012-11-26             NaN
2012-12-03    1.317542e+06
2012-12-10             NaN
2012-12-17             NaN
2012-12-24             NaN
2012-12-31    2.384847e+06
Freq: W-MON, Name: LineTotal, dtype: float64

Pandas har stöd för massor av olika frekvensberäkningar, följande tabell visar vad vi kan använda oss av.

<img src="./assets/images/ts_freq.png" width="600" align="left">

För att räkna på exempelvis procentuella förändringar över olika datumfönster har Pandas funktionalitet liknande `lead` och `lag` inom SQL. I Pandas fall heter funktionen `.shift()`. I dess enklaste form flyttas helt enkelt datat en rad bakåt eller framåt utan att justera indexet. Se exempel nedan.

In [79]:
print(ts.head())
print(ts.shift(1).head())

OrderDate
2011-05-31    4.893286e+05
2011-07-01    1.538408e+06
2011-08-01    1.165897e+06
2011-08-31    8.447210e+05
2011-10-01    2.324136e+06
Name: LineTotal, dtype: float64
OrderDate
2011-05-31             NaN
2011-07-01    4.893286e+05
2011-08-01    1.538408e+06
2011-08-31    1.165897e+06
2011-10-01    8.447210e+05
Name: LineTotal, dtype: float64


In [80]:
print(ts.head())
print(ts.shift(-1).head())

OrderDate
2011-05-31    4.893286e+05
2011-07-01    1.538408e+06
2011-08-01    1.165897e+06
2011-08-31    8.447210e+05
2011-10-01    2.324136e+06
Name: LineTotal, dtype: float64
OrderDate
2011-05-31    1.538408e+06
2011-07-01    1.165897e+06
2011-08-01    8.447210e+05
2011-08-31    2.324136e+06
2011-10-01    1.702945e+06
Name: LineTotal, dtype: float64


Med detta kan man enkelt göra deltaberäkningar.

In [81]:
df = pd.concat([ts, ts.shift(1)], axis=1)
df.columns = ['period', 'lastperiod']
df.head()

,period,lastperiod
OrderDate,,
2011-05-31,4.893286e+05,NaN
2011-07-01,1.538408e+06,4.893286e+05
2011-08-01,1.165897e+06,1.538408e+06
2011-08-31,8.447210e+05,1.165897e+06
2011-10-01,2.324136e+06,8.447210e+05


In [82]:
df['change'] = df['period'] - df['lastperiod']
df.head()

,period,lastperiod,change
OrderDate,,,
2011-05-31,4.893286e+05,NaN,NaN
2011-07-01,1.538408e+06,4.893286e+05,1.049080e+06
2011-08-01,1.165897e+06,1.538408e+06,-3.725112e+05
2011-08-31,8.447210e+05,1.165897e+06,-3.211761e+05
2011-10-01,2.324136e+06,8.447210e+05,1.479415e+06


Om vi istället för en enkel offset av raderna vill shifta datumindex kan vi göra detta genom att ange `freq=` som nedan.

In [83]:
ts.shift(1, freq='D').head()

OrderDate
2011-06-01    4.893286e+05
2011-07-02    1.538408e+06
2011-08-02    1.165897e+06
2011-09-01    8.447210e+05
2011-10-02    2.324136e+06
Name: LineTotal, dtype: float64

Om vi concatenerar med vår föregående dataframe ser vi att indexet har skiftats med en dag framåt. 

In [84]:
pd.concat([df, ts.shift(1, freq='D')], axis=1).head(10)

,period,lastperiod,change,LineTotal
OrderDate,,,,
2011-05-31,4.893286e+05,NaN,NaN,NaN
2011-06-01,NaN,NaN,NaN,4.893286e+05
2011-07-01,1.538408e+06,4.893286e+05,1.049080e+06,NaN
2011-07-02,NaN,NaN,NaN,1.538408e+06
2011-08-01,1.165897e+06,1.538408e+06,-3.725112e+05,NaN
2011-08-02,NaN,NaN,NaN,1.165897e+06
2011-08-31,8.447210e+05,1.165897e+06,-3.211761e+05,NaN
2011-09-01,NaN,NaN,NaN,8.447210e+05
2011-10-01,2.324136e+06,8.447210e+05,1.479415e+06,NaN


#### date parts
Om vi vill ta ut specifika attribut på våra datum erbjuder Pandas en uppsjö alternativ.

In [86]:
print(ts.index.year)
print(ts.index.month)
print(ts.index.dayofweek)

[2011 2011 2011 2011 2011 2011 2011 2012 2012 2012 2012 2012 2012 2012 2012
 2012 2012 2012 2012 2012 2013 2013 2013 2013 2013 2013 2013 2013 2013 2013
 2013 2013 2014 2014 2014 2014 2014 2014 2014 2014]
[ 5  7  8  8 10 10 12  1  1  2  3  4  5  6  7  8  9 10 11 12  1  2  3  4  5
  6  7  8  9 10 11 12  1  1  2  3  3  3  4  5]
[1 4 0 2 5 0 3 6 6 2 4 0 2 5 1 3 6 1 4 0 0 3 5 1 3 6 2 4 0 2 5 1 1 2 4 5 6
 0 2 3]


In [87]:
df['year'] = df.index.year

Vi kan enkelt lägga in dessa som egna kolumner i en dataframe för att kunna analyser de vidare.

In [88]:
df = pd.DataFrame(ts)
df['DayOfWeek'] = df.index.dayofweek
df.head()

,LineTotal,DayOfWeek
OrderDate,,
2011-05-31,4.893286e+05,1
2011-07-01,1.538408e+06,4
2011-08-01,1.165897e+06,0
2011-08-31,8.447210e+05,2
2011-10-01,2.324136e+06,5


In [89]:
df['LineTotal'] = df['LineTotal'].astype('int')

In [90]:
df.groupby('DayOfWeek')['LineTotal'].mean()

DayOfWeek
0    2088692
1    1708857
2    2060030
3    2092690
4    1482470
5    2535914
6    2127620
Name: LineTotal, dtype: int64

Det går också att gruppera direkt på attributen.

In [91]:
ts = ts.astype('int')

In [92]:
ts.groupby(ts.index.dayofweek).agg(['sum', 'mean'])

,sum,mean
0,14620849,2088692
1,10253146,1708857
2,14420214,2060030
3,10463450,2092690
4,7412351,1482470
5,12679570,2535914
6,10638103,2127620


#### rolling calculations
För att arbeta med rullande beräkningar har Pandas ett antal funktioner att tillgå. För att räkna ett rullande medelvärde över 3 observationer används funktionen `.rolling()` enligt nedan.

In [95]:
ts.rolling(3).mean()

OrderDate
2011-05-31             NaN
2011-07-01             NaN
2011-08-01    1.064544e+06
2011-08-31    1.183008e+06
2011-10-01    1.444917e+06
2011-10-31    1.623933e+06
2011-12-01    1.580065e+06
2012-01-01    1.438949e+06
2012-01-29    1.356395e+06
2012-02-29    1.412989e+06
2012-03-30    1.535765e+06
2012-04-30    1.384606e+06
2012-05-30    1.888203e+06
2012-06-30    2.332227e+06
2012-07-31    2.960079e+06
2012-08-30    2.762901e+06
2012-09-30    2.580443e+06
2012-10-30    2.347061e+06
2012-11-30    2.185523e+06
2012-12-31    1.962533e+06
2013-01-28    1.755447e+06
2013-02-28    1.938026e+06
2013-03-30    2.103328e+06
2013-04-30    2.244634e+06
2013-05-30    2.512016e+06
2013-06-30    2.958741e+06
2013-07-31    3.645855e+06
2013-08-30    3.518066e+06
2013-09-30    3.273385e+06
2013-10-30    3.094054e+06
2013-11-30    2.889019e+06
2013-12-31    2.627994e+06
2014-01-28    1.458109e+06
2014-01-29    1.814187e+06
2014-02-28    9.139937e+05
2014-03-01    1.648319e+06
2014-03-30    7.38

Om vi har data som har oregelbundna interval vill vi antagligen inte arbeta med rullande observationer utan rullande perioder. Pandas har inbyggt stöd för att ange valfri frekvens som i exemplet nedan då vi för varje observation vill räkna ut genomsnitt för de tre senaste månaderna.

In [99]:
ts

OrderDate
2011-05-31     489328
2011-07-01    1538408
2011-08-01    1165897
2011-08-31     844720
2011-10-01    2324135
2011-10-31    1702944
2011-12-01     713116
2012-01-01    1900788
2012-01-29    1455280
2012-02-29     882899
2012-03-30    2269116
2012-04-30    1001803
2012-05-30    2393689
2012-06-30    3601190
2012-07-31    2885359
2012-08-30    1802154
2012-09-30    3053816
2012-10-30    2185213
2012-11-30    1317541
2012-12-31    2384846
2013-01-28    1563955
2013-02-28    1865278
2013-03-30    2880752
2013-04-30    1987872
2013-05-30    2667423
2013-06-30    4220928
2013-07-31    4049215
2013-08-30    2284056
2013-09-30    3486885
2013-10-30    3511220
2013-11-30    1668952
2013-12-31    2703810
2014-01-28       1564
2014-01-29    2737187
2014-02-28       3230
2014-03-01    2204541
2014-03-30       7291
2014-03-31    3314519
2014-04-30       1284
2014-05-01    3415479
Name: LineTotal, dtype: int64

In [103]:
ts.resample('M').sum().rolling(3, min_periods=1).mean()

OrderDate
2011-05-31    4.893280e+05
2011-06-30    4.893280e+05
2011-07-31    1.013868e+06
2011-08-31    1.774512e+06
2011-09-30    1.774512e+06
2011-10-31    3.018848e+06
2011-11-30    4.027079e+06
2011-12-31    2.370098e+06
2012-01-31    2.034592e+06
2012-02-29    1.650694e+06
2012-03-31    2.169361e+06
2012-04-30    1.384606e+06
2012-05-31    1.888203e+06
2012-06-30    2.332227e+06
2012-07-31    2.960079e+06
2012-08-31    2.762901e+06
2012-09-30    2.580443e+06
2012-10-31    2.347061e+06
2012-11-30    2.185523e+06
2012-12-31    1.962533e+06
2013-01-31    1.755447e+06
2013-02-28    1.938026e+06
2013-03-31    2.103328e+06
2013-04-30    2.244634e+06
2013-05-31    2.512016e+06
2013-06-30    2.958741e+06
2013-07-31    3.645855e+06
2013-08-31    3.518066e+06
2013-09-30    3.273385e+06
2013-10-31    3.094054e+06
2013-11-30    2.889019e+06
2013-12-31    2.627994e+06
2014-01-31    2.370504e+06
2014-02-28    1.815264e+06
2014-03-31    2.756111e+06
2014-04-30    1.843622e+06
2014-05-31    2.98

Det finns otroligt mycket mer att gå igenom gällande både aggregeringar och tidsserier men vi har täckt in de viktigaste delarna och har förhoppningsvis byggt upp en grund att stå på för att utforska Pandas vidare. 

Det är ett mycket kraftfullt bibliotek och väldigt lätt att arbeta med efter man kommit över första tröskeln.